# Chapter 5: 支持向量机 (SVM) 实践 - 参考答案

## 实验概述

在本实验中，你将使用 **Wine 数据集**来学习支持向量机（SVM）的实现和应用。

## Part 1: 数据加载与探索

### 1.1 导入必要的库

In [ ]:
# ===== 预填充代码 =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

print("库导入成功！")

### 1.2 加载 Wine 数据集

In [ ]:
# ===== 预填充代码 =====
wine = load_wine()
X = wine.data
y = wine.target
feature_names = wine.feature_names
target_names = wine.target_names

print("数据集信息:")
print(f"样本数量: {X.shape[0]}")
print(f"特征数量: {X.shape[1]}")
print(f"类别数量: {len(target_names)}")
print(f"\n类别名称: {target_names}")

### 1.3 数据探索

In [ ]:
# ===== 参考答案 =====
# 创建 DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print("前10行数据:")
print(df.head(10))

print("\n每个类别的样本数量:")
print(df['target'].value_counts().sort_index())

### 1.4 数据标准化

In [ ]:
# ===== 预填充代码 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("标准化后的统计信息:")
print(f"均值: {X_scaled.mean(axis=0)[:5]}")
print(f"标准差: {X_scaled.std(axis=0)[:5]}")

### 1.5 划分训练集和测试集

In [ ]:
# ===== 预填充代码 =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")

## Part 2: 核函数实现

### 2.1 线性核函数

In [ ]:
# ===== 参考答案 =====
def linear_kernel(x1, x2):
    """
    线性核函数
    
    参数:
    x1: 第一个向量
    x2: 第二个向量
    
    返回:
    核函数值 (点积)
    """
    return np.dot(x1, x2)

# 测试
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
result = linear_kernel(x1, x2)
print(f"线性核测试: K({x1}, {x2}) = {result}")
# 预期输出: 32 (1*4 + 2*5 + 3*6 = 4 + 10 + 18 = 32)

### 2.2 RBF 核函数（高斯核）

In [ ]:
# ===== 参考答案 =====
def rbf_kernel(x1, x2, gamma=1.0):
    """
    RBF (高斯) 核函数
    
    参数:
    x1: 第一个向量
    x2: 第二个向量  
    gamma: 核参数
    
    返回:
    核函数值
    """
    # 计算欧氏距离平方
    distance_squared = np.sum((x1 - x2) ** 2)
    # 应用高斯函数
    return np.exp(-gamma * distance_squared)

# 测试 RBF 核函数
x1 = np.array([1, 2, 3])
x2 = np.array([1, 2, 3])  # 相同的向量
result_same = rbf_kernel(x1, x2, gamma=1.0)

x3 = np.array([4, 5, 6])
result_diff = rbf_kernel(x1, x3, gamma=0.1)

print(f"RBF 核测试 (相同向量): K(x, x) = {result_same:.6f}")
print(f"RBF 核测试 (不同向量): K(x1, x2) = {result_diff:.6f}")

## Part 3: 使用 scikit-learn 的 SVM

### 3.1 创建线性核 SVM

In [ ]:
# ===== 参考答案 =====
# 创建线性核 SVM 模型
svm_linear = svm.SVC(kernel='linear', C=1.0, random_state=42)

# 训练模型
svm_linear.fit(X_train, y_train)

# 预测
y_train_pred_linear = svm_linear.predict(X_train)
y_test_pred_linear = svm_linear.predict(X_test)

# 计算准确率
train_acc_linear = accuracy_score(y_train, y_train_pred_linear)
test_acc_linear = accuracy_score(y_test, y_test_pred_linear)

print(f"线性核 SVM:")
print(f"  训练集准确率: {train_acc_linear:.4f}")
print(f"  测试集准确率: {test_acc_linear:.4f}")
print(f"  支持向量数量: {len(svm_linear.support_vectors_)}")

### 3.2 创建 RBF 核 SVM

In [ ]:
# ===== 参考答案 =====
# 创建 RBF 核 SVM 模型
svm_rbf = svm.SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# 训练模型
svm_rbf.fit(X_train, y_train)

# 预测
y_train_pred_rbf = svm_rbf.predict(X_train)
y_test_pred_rbf = svm_rbf.predict(X_test)

# 计算准确率
train_acc_rbf = accuracy_score(y_train, y_train_pred_rbf)
test_acc_rbf = accuracy_score(y_test, y_test_pred_rbf)

print(f"RBF 核 SVM:")
print(f"  训练集准确率: {train_acc_rbf:.4f}")
print(f"  测试集准确率: {test_acc_rbf:.4f}")
print(f"  支持向量数量: {len(svm_rbf.support_vectors_)}")

### 3.3 比较不同核函数

In [ ]:
# ===== 预填充代码 =====
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
results = []

for kernel in kernels:
    model = svm.SVC(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    n_sv = len(model.support_vectors_)
    
    results.append({
        'kernel': kernel,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'n_support_vectors': n_sv
    })

results_df = pd.DataFrame(results)
print("不同核函数的比较:")
print(results_df)

## Part 4: 超参数调优

### 4.1 网格搜索调优

In [ ]:
# ===== 参考答案 =====
# 定义参数网格
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# 创建 GridSearchCV 对象
grid_search = GridSearchCV(
    svm.SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 获取最佳参数和分数
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("网格搜索结果:")
print(f"最佳参数: {best_params}")
print(f"最佳交叉验证分数: {best_score:.4f}")

### 4.2 使用最佳模型进行预测

In [ ]:
# ===== 参考答案 =====
# 获取最佳模型
best_model = grid_search.best_estimator_

# 在测试集上预测
y_pred_best = best_model.predict(X_test)

# 计算测试集准确率
test_acc_best = accuracy_score(y_test, y_pred_best)

print(f"最佳模型在测试集上的准确率: {test_acc_best:.4f}")
print(f"\n分类报告:")
print(classification_report(y_test, y_pred_best, target_names=target_names))

## Part 5: 模型评估与可视化

### 5.1 混淆矩阵

In [ ]:
# ===== 参考答案 =====
# 计算混淆矩阵
cm = confusion_matrix(y_test, y_pred_best)

print("混淆矩阵:")
print(cm)

# 可视化混淆矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('预测类别')
plt.ylabel('真实类别')
plt.title('混淆矩阵')
plt.show()

### 5.2 支持向量分析

In [ ]:
# ===== 预填充代码 =====
model = best_model

print("支持向量分析:")
print(f"总支持向量数量: {len(model.support_vectors_)}")

if hasattr(model, 'n_support_'):
    for i, count in enumerate(model.n_support_):
        print(f"  类别 {i} ({target_names[i]}): {count} 个支持向量")

print(f"\n支持向量索引 (前10个): {model.support_[:10]}")

### 5.3 可视化支持向量和决策边界（2D 投影）

In [ ]:
# ===== 预填充代码 =====
# 选择两个特征进行可视化
feature_idx1 = 0  # alcohol
feature_idx2 = 1  # malic_acid

X_train_2d = X_train[:, [feature_idx1, feature_idx2]]
X_test_2d = X_test[:, [feature_idx1, feature_idx2]]

# 训练 2D 模型
model_2d = svm.SVC(kernel='rbf', C=best_params['C'], 
                   gamma=best_params['gamma'], random_state=42)
model_2d.fit(X_train_2d, y_train)

# 创建网格
x_min, x_max = X_scaled[:, feature_idx1].min() - 1, X_scaled[:, feature_idx1].max() + 1
y_min, y_max = X_scaled[:, feature_idx2].min() - 1, X_scaled[:, feature_idx2].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

Z = model_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# 绘图
plt.figure(figsize=(12, 5))

# 左图: 训练集
plt.subplot(1, 2, 1)
colors = ['red', 'green', 'blue']
markers = ['o', 's', '^']
for i in range(3):
    plt.scatter(X_train_2d[y_train == i, 0], X_train_2d[y_train == i, 1],
                c=colors[i], marker=markers[i], label=f'类别 {i}', alpha=0.6)

plt.scatter(model_2d.support_vectors_[:, 0], model_2d.support_vectors_[:, 1],
            s=100, facecolors='none', edgecolors='yellow', linewidths=2,
            label='支持向量')

plt.contourf(xx, yy, Z, alpha=0.2, levels=2)
plt.xlabel(feature_names[feature_idx1])
plt.ylabel(feature_names[feature_idx2])
plt.title('训练集 - 支持向量与决策边界')
plt.legend()

# 右图: 测试集
plt.subplot(1, 2, 2)
for i in range(3):
    plt.scatter(X_test_2d[y_test == i, 0], X_test_2d[y_test == i, 1],
                c=colors[i], marker=markers[i], label=f'类别 {i}', alpha=0.6)

plt.contourf(xx, yy, Z, alpha=0.2, levels=2)
plt.xlabel(feature_names[feature_idx1])
plt.ylabel(feature_names[feature_idx2])
plt.title('测试集 - 决策边界')
plt.legend()

plt.tight_layout()
plt.show()

### 5.4 特征重要性分析

In [ ]:
# ===== 参考答案 =====
# 获取支持向量的索引
support_indices = best_model.support_

# 获取支持向量的特征值（使用标准化后的数据）
support_vectors = X_train[support_indices]

# 计算每个特征在支持向量中的方差
feature_importance = np.var(support_vectors, axis=0)

# 创建 DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("特征重要性（基于支持向量的方差）:")
print(importance_df)

# 可视化
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('重要性 (方差)')
plt.ylabel('特征')
plt.title('特征重要性分析')
plt.tight_layout()
plt.show()

## Part 6: 挑战练习

### 6.1 多项式核 SVM

In [ ]:
# ===== 参考答案 =====
# 比较不同 degree 的多项式核
degrees = [2, 3, 4, 5]
poly_train_accs = []
poly_test_accs = []

for degree in degrees:
    model = svm.SVC(kernel='poly', degree=degree, random_state=42)
    model.fit(X_train, y_train)
    
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    
    poly_train_accs.append(train_acc)
    poly_test_accs.append(test_acc)
    
    print(f"Degree {degree}: 训练={train_acc:.4f}, 测试={test_acc:.4f}")

# 绘制结果
plt.figure(figsize=(10, 5))
plt.plot(degrees, poly_train_accs, 'o-', label='训练集准确率', linewidth=2)
plt.plot(degrees, poly_test_accs, 's-', label='测试集准确率', linewidth=2)
plt.xlabel('多项式 Degree')
plt.ylabel('准确率')
plt.title('多项式核不同 Degree 的性能比较')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 6.2 交叉验证

In [ ]:
# ===== 参考答案 =====
# 使用最佳参数进行 5 折交叉验证
cv_scores = cross_val_score(
    best_model, 
    X_scaled, 
    y, 
    cv=5, 
    scoring='accuracy'
)

print("5 折交叉验证结果:")
print(f"每折分数: {cv_scores}")
print(f"平均分数: {cv_scores.mean():.4f}")
print(f"标准差: {cv_scores.std():.4f}")

# 可视化交叉验证结果
plt.figure(figsize=(8, 5))
plt.bar(range(1, 6), cv_scores, alpha=0.7, edgecolor='black')
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', 
            label=f'平均分: {cv_scores.mean():.4f}')
plt.xlabel('折数')
plt.ylabel('准确率')
plt.title('5 折交叉验证结果')
plt.xticks(range(1, 6))
plt.legend()
plt.ylim(0.8, 1.0)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 总结

在本实验中，我们学习了：

1. **SVM 基本原理**: 最大间隔分类，支持向量决定决策边界
2. **核函数**: 线性核、RBF 核、多项式核的实现
3. **超参数调优**: C 参数控制正则化，gamma 控制 RBF 核的影响范围
4. **模型评估**: 混淆矩阵、准确率、交叉验证
5. **可视化**: 支持向量和决策边界的可视化

### 关键要点

- **核函数选择**: 线性可分数据优先使用线性核，非线性数据使用 RBF 核
- **数据标准化**: SVM 对特征尺度敏感，必须进行标准化
- **C 参数**: C 越大，模型越复杂，容易过拟合；C 越小，模型越简单，可能欠拟合
- **gamma 参数**: gamma 越大，决策边界越复杂；gamma 越小，决策边界越平滑